In [2]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

setup geometry

In [199]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "in"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.2))
mesh.Curve(3);
#Draw(mesh)

setup FEM space

In [200]:
# viscosity
nu = 0.001
# Order of spaces
k = 2
# H1 vs VectorH1 -> vector field?!
V = VectorH1(mesh,order=k, dirichlet="top|bot|cyl|in|out")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q]) # X = [V,V,Q] (without VectorH1)

setup bilinear form

In [201]:
u,p = X.TrialFunction()
v,q = X.TestFunction()

a = BilinearForm(X)
a += (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a.Assemble()

setup boundary conditions (?)

In [202]:
gfu = GridFunction(X)
# setup flow condition
uinf = 0.001
uin = CoefficientFunction((uinf,0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("in|top|bot|out"))

velocity = CoefficientFunction(gfu.components[0])
scene = Draw(velocity, mesh, "vel")
print("->")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

->


solve equation

In [203]:
res = gfu.vec.CreateVector()
res.data = -a.mat * gfu.vec
inv = a.mat.Inverse(X.FreeDofs())
gfu.vec.data += inv * res
scene.Redraw()

In [204]:
def solveStokes():
    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(X.FreeDofs())
    gfu.vec.data += inv * res
    scene.Redraw()
    return res

Drag/ "dissipated energy"

$J(\Omega ) = \frac{1}{2} \int_\Omega Du : Du dx$

In [205]:
vel = gfu.components[0].vec.data #gfu.vec.data
drag = 0.5*InnerProduct(vel,vel)
print(drag)
# we kinda just need the derivative of u, not this
t = 0.0005310368441135379
print(t-drag)

0.00040430826264929763
0.00012672858146424025


maybe try this to get calculated u vector: f += CoefficientFunction([0, 0, 1])*v*dx

### working drag calculation
taken from navier stokes, is more complicated than needed

In [219]:
drag_test = GridFunction(X)
drag_test.components[0].Set(CoefficientFunction((-1.0,0)), definedon=mesh.Boundaries("cyl"))

res = solveStokes()
drag_val = InnerProduct(res, drag_test.vec)
print(drag_val)

-0.021995659668388713
V0 (-3.0, -2.0)
V1 (3.0, -2.0)
V2 (3.0, 2.0)
V3 (-3.0, 2.0)
V4 (0.0, -0.5)
V5 (0.5, 0.0)
V6 (0.0, 0.5)
V7 (-0.5, 0.0)
V8 (-2.8000000000000003, -2.0)
V9 (-2.5999999999999996, -2.0)
V10 (-2.4, -2.0)
V11 (-2.2, -2.0)
V12 (-2.0, -2.0)
V13 (-1.8, -2.0)
V14 (-1.6, -2.0)
V15 (-1.4, -2.0)
V16 (-1.1999999999999997, -2.0)
V17 (-1.0, -2.0)
V18 (-0.7999999999999998, -2.0)
V19 (-0.6000000000000001, -2.0)
V20 (-0.3999999999999999, -2.0)
V21 (-0.20000000000000018, -2.0)
V22 (0.0, -2.0)
V23 (0.20000000000000018, -2.0)
V24 (0.3999999999999999, -2.0)
V25 (0.6000000000000005, -2.0)
V26 (0.7999999999999998, -2.0)
V27 (1.0, -2.0)
V28 (1.2000000000000002, -2.0)
V29 (1.4000000000000004, -2.0)
V30 (1.5999999999999996, -2.0)
V31 (1.7999999999999998, -2.0)
V32 (2.0, -2.0)
V33 (2.2, -2.0)
V34 (2.4000000000000004, -2.0)
V35 (2.5999999999999996, -2.0)
V36 (2.799999999999999, -2.0)
V37 (3.0, -1.8)
V38 (3.0, -1.5999999999999999)
V39 (3.0, -1.4000000000000001)
V40 (3.0, -1.2)
V41 (3.0, -1.0)
V42

### get surface area of mesh
(without ball), should stay constant

In [218]:
surf_test = GridFunction(V)
surf_test.Set(CoefficientFunction((1,1))) # , definedon=mesh.Boundaries("cyl")
#print(surf_test.vec/mesh.nedge) #*mesh.nface

       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
       1
 

In [248]:
#for v in mesh.vertices:
#    print (v, v.point)
nr = []
for e in mesh.Boundaries("cyl").Elements():
    for i in e.vertices:
        nr.append(i.nr)
nr = list(set(nr))    
print(nr)

[4, 5, 6, 7, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]


### Shape derivative
$S_1 = \biggl(\frac{1}{2}Du : Du - p div(u) \biggr) I_2 + Du^T p - Du^T Du.$

In [208]:
l = (0.5*InnerProduct(grad(u),grad(u))-p*div(v))*Id(2)
r = grad(u).trans*p-grad(u).trans*grad(u)
S = l+r

In [209]:
r = X.TrialFunction()
s = X.TestFunction()
gf = GridFunction(X)

In [ ]:
# gf, gfp: solution to state&adjoint eq

In [1]:
def Cost(u):
    return 0.5*InnerProduct(grad(u),grad(u))*dx

def Equation(u,w):
    return (InnerProduct(grad(u),grad(v))+div(u)*q-div(v)*p)*dx

G_pde = Cost(gfu) * Equation(gfu, gfp)

W = V.TestFunction() # X. or V.
dJOmega = LinearForm(VEC)
dfOmega += G_pdre.DiffShape(W)

NameError: name 'gfu' is not defined

In [71]:
"""
grad_S = CoefficientFunction((S.Diff(x),S.Diff(y)))
VEC = H1(mesh, order=1, dim=2)
# grid function for deformation field
gfset = GridFunction(VEC)
gfX = GridFunction(VEC)
scene2 = Draw(gfset)

# Test and trial functions
PHI, PSI = VEC.TnT()

# shape derivative
dJOmega = LinearForm(VEC)
dJOmega += (div(PSI)*f + InnerProduct(grad_f, PSI) )*dx
"""

'\ngrad_S = CoefficientFunction((S.Diff(x),S.Diff(y)))\nVEC = H1(mesh, order=1, dim=2)\n# grid function for deformation field\ngfset = GridFunction(VEC)\ngfX = GridFunction(VEC)\nscene2 = Draw(gfset)\n\n# Test and trial functions\nPHI, PSI = VEC.TnT()\n\n# shape derivative\ndJOmega = LinearForm(VEC)\ndJOmega += (div(PSI)*f + InnerProduct(grad_f, PSI) )*dx\n'